In [1]:
from pyqubo import Spin, Binary, Array
import math
import numpy as np


def basis_to_gram_matrix(basis):
    if not isinstance(basis, np.ndarray):
        raise ValueError("Basis must be a numpy array")
    
    return basis @ basis.T


def gram_to_quadratic_form(gram_matrix):
    x, y = gram_matrix.shape
    if x != y:
        raise ValueError("Gram matrix must be square")
    
    # Diagonal terms
    diag = {}
    for i in range(x):
        diag[i] = gram_matrix[i, i]
    
    # Off-diagonal terms
    off_diag = {}
    for i in range(x):
        for j in range(i + 1, y):
            off_diag[(i, j)] = gram_matrix[i, j] + gram_matrix[j, i]
    
    return diag, off_diag


def substitution_array(variable_name, num_variables, bounds):
    # Handle bounds input
    if isinstance(bounds, (int, float)):
        bounds = [bounds] * num_variables
    elif len(bounds) != num_variables:
        raise ValueError("Bounds list must match number of variables")
    
    substitutions = []
    
    for i, bound in enumerate(bounds):
        num_bits = math.floor(math.log2(2 * bound)) + 1
        
        # Create binary array for this variable
        binary_vars = Array.create(f"{variable_name}_{i}", shape=num_bits, vartype='BINARY')
        
        # Coefficients for the encoding
        coeffs = [2**j for j in range(num_bits - 1)]
        last_coeff = 2 * bound + 1 - 2**math.floor(math.log2(2 * bound))
        coeffs.append(last_coeff)
        
        # Build the substitution expression
        sub = -bound
        for j, coeff in enumerate(coeffs):
            sub += coeff * binary_vars[j]
        
        substitutions.append(sub)
    print(substitutions)
    return substitutions

def substitution_array_for_nonzero(variable_name, num_variables, bounds):
    if isinstance(bounds, (int, float)):
        bounds = [bounds] * num_variables
    elif len(bounds) != num_variables:
        raise ValueError("Bounds list must match number of variables")
    
    substitutions = []
    
    for i, bound in enumerate(bounds):
        # Calculate number of bits needed
        num_bits = math.floor(math.log2(bound - 1)) + 1
        print(f"Nium bits = {num_bits} for bound {bound}")
        
        # Create binary array for this variable
        binary_vars = Array.create(f"{variable_name}_{i}", shape=num_bits + 2, vartype='BINARY')
        
        # x_i = -a_i + ξ_i + ω_i(a + 1) + Σ(2^j * x_ij) + (a_i - 2^⌊log(a_i-1)⌋) * x_i,⌊log(a-1)⌋
        # special encoding hehe
        
        # Build the substitution expression
        sub = -bound  # -a_i term
        
        # ξ_i term (first binary variable)
        sub += binary_vars[0]
        
        # ω_i(a + 1) term (second binary variable)
        sub += binary_vars[1] * (bound + 1)
        
        # Σ(2^j * x_ij) terms for j from 0 to ⌊log(a_i-1)⌋-1
        for j in range(num_bits-1):
            sub += (2**j) * binary_vars[j + 2]
        
        # Final term: (a_i - 2^⌊log(a_i-1)⌋) * x_i,⌊log(a-1)⌋
        if num_bits > 0:
            final_coeff = bound - (2**math.floor(math.log2(bound - 1)))
            sub += final_coeff * binary_vars[num_bits + 1]
            print(f"Appending substitution for {final_coeff} * {binary_vars[num_bits + 1]}")
        substitutions.append(sub)
    print(substitutions)
    return substitutions

def gram_to_hamiltonian(gram_matrix, bounds, variable_name='x'):
    n = gram_matrix.shape[0]
    
    # Convert Gram matrix to quadratic form
    diagonal_terms, off_diagonal_terms = gram_to_quadratic_form(gram_matrix)
    
    # Create substitutions - special non-zero substitution only for x_0
    substitutions = []
    
    for i in range(n):
        # if False: USE THIS IF YOU WANT TO USE THE NORMAL SUBSTITUTION
        if i == 0:
            # Use special non-zero substitution for x_0
            # Extract the bound for this variable and ensure it's a scalar
            if isinstance(bounds, (list, np.ndarray)) and len(bounds) > 1:
                bound_for_i = int(bounds[0])  # Convert to Python int
            else:
                bound_for_i = int(bounds) if isinstance(bounds, np.ndarray) else bounds
            special_sub = substitution_array_for_nonzero(f"{variable_name}_control", 1, bound_for_i)
            substitutions.append(special_sub[0])
        else:
            # Use regular substitution for all other variables
            # Extract the bound for this variable and ensure it's a scalar
            if isinstance(bounds, (list, np.ndarray)) and len(bounds) > 1:
                bound_for_i = int(bounds[i])  # Convert to Python int
            else:
                bound_for_i = int(bounds) if isinstance(bounds, np.ndarray) else bounds
            # FIX: Include the variable index in the variable name
            regular_sub = substitution_array(f"{variable_name}_{i}", 1, bound_for_i)
            substitutions.append(regular_sub[0])
    
    # Build the Hamiltonian
    hamiltonian = 0
    
    
    #IMPORTANT note, I AM ENCODING x_0 AS THE PENALITY INSETAD OF PENALIZING x_0 * x_1 * x_2 * ... * x_n, refer to original paper
    #if you want to penalize the product of all variables instead
    
    # Add penalty for the first binary variable of x_0 (x_control_0) = 1
    # This corresponds to the ξ_0 term in the non-zero encoding
    if len(substitutions) > 0:  # Ensure x_0 exists
        # Create the first binary variable for the penalty
        penalty_var = Binary(f"{variable_name}_control_0[0]")  # First binary var of first special variable
        # hamiltonian += 5 * penalty_var  # Penalty of 5 when this binary variable = 1
        # testing_var = Binary(f"{variable_name}_0[0]")  # Second binary var of first special variable
        ## CHANGE THIS TO USE PENALITY VAR, UNCOMMENT THE ABOVE LINES
    
    # Add diagonal terms (quadratic terms like ax²)
    for i, coeff in diagonal_terms.items():
        hamiltonian += coeff * (substitutions[i] ** 2)
    
    # Add off-diagonal terms (cross terms like bxy)
    for (i, j), coeff in off_diagonal_terms.items():
        hamiltonian += coeff * substitutions[i] * substitutions[j]
    
    # Compile the model
    model = hamiltonian.compile()
    return model


def generate_qubo_from_gram_matrix_array(gram_matrix, bounds, variable_name='x'):
    model = gram_to_hamiltonian(gram_matrix, bounds, variable_name)
    qubo_dict, offset = model.to_qubo()
    
    return qubo_dict, offset
def generate_ising_from_gram_matrix_array(gram_matrix, bounds, variable_name='x'):
    model = gram_to_hamiltonian(gram_matrix, bounds, variable_name)
    linear, quadratic, offset = model.to_ising()
    
    return linear , quadratic, offset

def basis_to_ising(basis, bounds, variable_name='x'):
    gram_matrix = basis_to_gram_matrix(basis)
    return generate_ising_from_gram_matrix_array(gram_matrix, bounds, variable_name)

In [2]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp
import re

def ising_to_sparse_pauli_op(linear_terms, quadratic_terms, offset):
    """
    Convert Ising model terms to SparsePauliOp format.
    
    Args:
        linear_terms: dict with variable names as keys, coefficients as values
        quadratic_terms: dict with (var1, var2) tuples as keys, coefficients as values  
        offset: constant term
    
    Returns:
        SparsePauliOp object
    """
    
    # Extract all variable names and map them to qubit indices
    all_vars = set()
    for var in linear_terms.keys():
        all_vars.add(var)
    for var_pair in quadratic_terms.keys():
        all_vars.update(var_pair)
    
    # Sort variables to ensure consistent ordering
    sorted_vars = sorted(list(all_vars))
    
    print("Sorted variables:", sorted_vars)
    
    # Create mapping from variable name to qubit index
    var_to_qubit = {var: i for i, var in enumerate(sorted_vars)}
    n_qubits = len(sorted_vars)

    
    pauli_list = []
    
    # Add constant term (identity operator)
    if offset != 0:
        identity_string = 'I' * n_qubits
        pauli_list.append([identity_string, offset])
    
    # Add linear terms (single Z operators)
    for var, coeff in linear_terms.items():
        pauli_string = ['I'] * n_qubits
        qubit_idx = var_to_qubit[var]
        pauli_string[qubit_idx] = 'Z'
        pauli_list.append([''.join(pauli_string), coeff])
    
    # Add quadratic terms (ZZ operators)
    for (var1, var2), coeff in quadratic_terms.items():
        pauli_string = ['I'] * n_qubits
        qubit_idx1 = var_to_qubit[var1]
        qubit_idx2 = var_to_qubit[var2]
        pauli_string[qubit_idx1] = 'Z'
        pauli_string[qubit_idx2] = 'Z'
        pauli_list.append([''.join(pauli_string), coeff])
    
    return SparsePauliOp.from_list(pauli_list)

basis = np.array([[-5, -2, 2], [-2, 5, 7], [6, -1, 6]])
bound = np.array([3,3,3])
ising = basis_to_ising(basis, bound)


print("Raw ising output:")
print(ising)
print()

# Unpack the result
linear_terms, quadratic_terms, offset = ising

# DEBUG: Print the linear and quadratic terms
print("Linear terms:", linear_terms)
print("Quadratic terms:", list(quadratic_terms.keys()))  # Show keys
print("Offset:", offset)
print()

# Convert to SparsePauliOp
ising_op =  ising_to_sparse_pauli_op(linear_terms, quadratic_terms, offset)
print("SparsePauliOp:") #comment this out if dont need debug
print(ising_op)

Nium bits = 2 for bound 3
Appending substitution for 1 * Binary('x_control_0[3]')
[((1.000000 * Binary('x_control_0[3]')) + (1.000000 * Binary('x_control_0[2]')) + (Binary('x_control_0[1]') * 4.000000) + -3.000000 + Binary('x_control_0[0]'))]
[((3.000000 * Binary('x_1_0[2]')) + (2.000000 * Binary('x_1_0[1]')) + -3.000000 + (1.000000 * Binary('x_1_0[0]')))]
[((3.000000 * Binary('x_2_0[2]')) + (2.000000 * Binary('x_2_0[1]')) + -3.000000 + (1.000000 * Binary('x_2_0[0]')))]
Raw ising output:
({'x_2_0[1]': -16.0, 'x_1_0[1]': 14.0, 'x_2_0[0]': -8.0, 'x_1_0[0]': 7.0, 'x_1_0[2]': 21.0, 'x_2_0[2]': -24.0, 'x_control_0[0]': 16.5, 'x_control_0[1]': 66.0, 'x_control_0[2]': 16.5, 'x_control_0[3]': 16.5}, {('x_1_0[0]', 'x_1_0[2]'): 117.0, ('x_1_0[1]', 'x_control_0[1]'): 56.0, ('x_1_0[0]', 'x_2_0[0]'): 12.5, ('x_1_0[0]', 'x_2_0[1]'): 25.0, ('x_1_0[0]', 'x_1_0[1]'): 78.0, ('x_1_0[0]', 'x_2_0[2]'): 37.5, ('x_1_0[0]', 'x_control_0[0]'): 7.0, ('x_1_0[0]', 'x_control_0[1]'): 28.0, ('x_1_0[1]', 'x_1_0[2]')

In [7]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from scipy.optimize import minimize
import matplotlib.pyplot as plt

def create_controlled_mixer(num_qubits, control_qubit=0):
    """
    Create controlled mixer where control_qubit controls RX on all others
    RX happens when control qubit is |0⟩
    """
    pauli_terms = []
    
    # For each target qubit (all except the control)
    for target_idx in range(num_qubits):
        if target_idx == control_qubit:
            continue  # Skip the control qubit itself
            
        # X_target term (from (I + Z)/2 * X = (X + ZX)/2)
        x_term = ["I"] * num_qubits
        x_term[target_idx] = "X"
        x_string = "".join(x_term)
        pauli_terms.append((x_string, 0.5))
        
        # Z_control * X_target term (positive for control=|0⟩)
        zx_term = ["I"] * num_qubits
        zx_term[control_qubit] = "Z"  # Z on control qubit
        zx_term[target_idx] = "X"     # X on target
        zx_string = "".join(zx_term)
        pauli_terms.append((zx_string, 0.5))  # Positive for control=|0⟩
    
    return SparsePauliOp.from_list(pauli_terms)

def create_manual_controlled_qaoa_ansatz(cost_operator, control_qubit=6, reps=1):
    """
    Create manual controlled QAOA ansatz that works with optimization
    Uses the bit-flip trick: X-control-X for control=|0⟩
    """
    num_qubits = cost_operator.num_qubits
    qc = QuantumCircuit(num_qubits)
    
    # Create parameter arrays
    gamma_params = [Parameter(f'γ[{i}]') for i in range(reps)]
    beta_params = [Parameter(f'β[{i}]') for i in range(reps)]
    
    print(f"Creating manual controlled QAOA ansatz:")
    print(f"  - Control qubit: q{control_qubit} (activates when |0⟩)")
    print(f"  - Target qubits: {[i for i in range(num_qubits) if i != control_qubit]}")
    print(f"  - QAOA layers: {reps}")
    
    # 1. Initialize all qubits in superposition
    qc.h(range(num_qubits))
    qc.barrier(label='Initial |+⟩')
    
    for rep in range(reps):
        # 2. Cost layer - apply the actual Hamiltonian evolution
        # We'll approximate this with the key terms from your ising_op
        
        # Single qubit terms (RZ gates)
        cost_coeffs = []
        for i, (pauli_str, coeff) in enumerate(zip(cost_operator.paulis, cost_operator.coeffs)):
            pauli_str = str(pauli_str)
            # Count non-identity terms
            non_identity_count = sum(1 for p in pauli_str if p != 'I')
            
            if non_identity_count == 1:  # Single qubit term
                qubit_idx = pauli_str.index('Z')
                qc.rz(2 * gamma_params[rep] * coeff.real, qubit_idx)
            elif non_identity_count == 2:  # Two qubit term
                z_positions = [i for i, p in enumerate(pauli_str) if p == 'Z']
                if len(z_positions) == 2:
                    qc.rzz(2 * gamma_params[rep] * coeff.real, z_positions[0], z_positions[1])
        
        qc.barrier(label=f'Cost {rep+1}')
        
        # 3. Controlled Mixer layer - the key innovation!
        # Use bit-flip trick: X-control-X makes control activate when |0⟩
        
        qc.x(control_qubit)  # Flip control qubit
        qc.barrier(label='Flip Control')
        
        # Apply controlled-RX gates (now activate when original state was |0⟩)
        for target in range(num_qubits):
            if target != control_qubit:
                qc.crx(2 * beta_params[rep], control_qubit, target)
        
        qc.barrier(label='Controlled RX')
        qc.x(control_qubit)  # Flip control qubit back
        qc.barrier(label='Flip Back')
    
    print(f"Manual ansatz created with {len(qc.parameters)} parameters")
    return qc

def create_qaoa_circuit(cost_operator, reps=1, use_controlled_mixer=False, control_qubit=0, use_manual=False):
    """
    MODIFIED: Added manual controlled circuit option
    """
    
    if use_manual and use_controlled_mixer:
        print(f"Using MANUAL controlled mixer: q{control_qubit} controls all other qubits (activates when q{control_qubit}=|0⟩)")
        return create_manual_controlled_qaoa_ansatz(cost_operator, control_qubit, reps)
    
    elif use_controlled_mixer:
        # Use QAOAAnsatz with custom mixer (the messy decomposition version)
        mixer_op = create_controlled_mixer(cost_operator.num_qubits, control_qubit)
        print(f"Using QAOAAnsatz controlled mixer: q{control_qubit} controls all other qubits (activates when q{control_qubit}=|0⟩)")
        qaoa_custom = QAOAAnsatz(cost_operator=cost_operator, mixer_operator=mixer_op, reps=reps, flatten=True)
        qaoa_custom.draw('mpl')
        plt.show()
        return qaoa_custom
    else:
        print("Using standard QAOA mixer")
        return QAOAAnsatz(
            cost_operator=cost_operator,
            reps=reps,
            flatten=True
        )

def create_sampling_circuit(qaoa_ansatz, parameters):
    """Create circuit with measurements for final sampling"""
    bound_circuit = qaoa_ansatz.assign_parameters(parameters)
    bound_circuit.measure_all()
    return bound_circuit

def objective_function_fixed(parameters, qaoa_ansatz, cost_operator):
    """Fixed objective function using StatevectorEstimator"""
    estimator = StatevectorEstimator()
    
    # Handle layout for hardware compatibility
    if hasattr(qaoa_ansatz, 'layout') and qaoa_ansatz.layout is not None:
        isa_hamiltonian = cost_operator.apply_layout(qaoa_ansatz.layout)
    else:
        isa_hamiltonian = cost_operator
    
    # Use the pub format
    pub = (qaoa_ansatz, isa_hamiltonian, parameters)
    job = estimator.run([pub])
    results = job.result()[0]
    
    return float(results.data.evs)

def run_qaoa_optimization(cost_operator, reps=1, max_iterations=500, use_controlled_mixer=False, control_qubit=0, use_manual=False):
    """
    MODIFIED: Added manual controlled circuit option
    """
    
    # Create QAOA ansatz
    qaoa_ansatz = create_qaoa_circuit(cost_operator, reps=reps, 
                                    use_controlled_mixer=use_controlled_mixer, 
                                    control_qubit=control_qubit,
                                    use_manual=use_manual)
    
    print(f"QAOA circuit has {qaoa_ansatz.num_parameters} parameters")
    print(f"Circuit depth: {qaoa_ansatz.depth()}")
    print(f"Number of qubits: {qaoa_ansatz.num_qubits}")
    
    # Show circuit visualization for manual version
    if use_manual and use_controlled_mixer:
        print("Manual controlled circuit created! Drawing...")
        fig = qaoa_ansatz.draw('mpl', style='iqp')
        # plt.title(f'Manual Controlled QAOA: q{control_qubit} controls all (activates when |0⟩)')
        # plt.tight_layout()
        # plt.show()
        
        # Also show text version
        print("Text representation:")
        print(qaoa_ansatz.draw())
    
    # Better parameter initialization for QAOA
    np.random.seed(69)
    initial_params = np.random.uniform(0, 0.5, qaoa_ansatz.num_parameters)
    print(f"Initial parameters: {initial_params}")
    
    # Test objective function
    print("Testing objective function...")
    initial_energy = objective_function_fixed(initial_params, qaoa_ansatz, cost_operator)
    print(f"Initial energy: {initial_energy:.3f}")
    
    # Run optimization with multiple methods if needed
    print("Starting optimization...")
    
    # Try COBYLA first (good for QAOA)
    result = minimize(
        objective_function_fixed,
        initial_params,
        args=(qaoa_ansatz, cost_operator),
        method='COBYLA',
        options={'maxiter': max_iterations, 'disp': True}
    )
    
    # If COBYLA didn't converge, try Powell
    if not result.success and result.nfev >= max_iterations * 0.8:
        print("\nCOBYLA hit max iterations, trying Powell method...")
        result = minimize(
            objective_function_fixed,
            result.x,  # Start from COBYLA result
            args=(qaoa_ansatz, cost_operator),
            method='Powell',
            options={'maxiter': max_iterations//2, 'disp': True}
        )
    
    print(f"\nOptimization completed!")
    print(f"Success: {result.success}")
    print(f"Optimal energy: {result.fun:.3f}")
    print(f"Energy improvement: {initial_energy - result.fun:.3f}")
    print(f"Message: {result.message}")
    
    return result, qaoa_ansatz

def sample_final_results(qaoa_ansatz, optimal_params, shots=1000):
    """Sample from the optimized quantum state"""
    
    # Create sampling circuit with measurements
    sampling_circuit = create_sampling_circuit(qaoa_ansatz, optimal_params)
    
    # Use StatevectorSampler for sampling
    sampler = StatevectorSampler()
    job = sampler.run([sampling_circuit], shots=shots)
    counts = job.result()[0].data.meas.get_counts()
    
    return counts

def evaluate_bitstring_cost(bitstring, cost_operator):
    """Evaluate the actual cost of a solution bitstring"""
    z_values = [1 if bit == '0' else -1 for bit in bitstring]
    
    cost = 0
    for pauli_string, coeff in zip(cost_operator.paulis, cost_operator.coeffs):
        term_value = 1
        for i, pauli in enumerate(str(pauli_string)):
            if pauli == 'Z':
                term_value *= z_values[i]
        cost += coeff.real * term_value
    
    return cost

def complete_qaoa_workflow(ising_op, reps=1, use_controlled_mixer=False, control_qubit=0, use_manual=False):
    """
    MODIFIED: Added manual controlled circuit option
    Complete QAOA workflow with controlled mixer support
    """
    
    print(f"\n{'='*60}")
    print(f"QAOA WORKFLOW")
    print(f"{'='*60}")
    print(f"Configuration:")
    print(f"  - QAOA layers (reps): {reps}")
    print(f"  - Use controlled mixer: {use_controlled_mixer}")
    print(f"  - Use manual implementation: {use_manual}")
    if use_controlled_mixer:
        print(f"  - Control qubit: q{control_qubit} (x_control_0[0])")
        print(f"  - Control logic: RX rotations when q{control_qubit} = |0⟩")
    print()
    
    # Run optimization
    result, qaoa_ansatz = run_qaoa_optimization(ising_op, reps=reps, max_iterations=500,
                                              use_controlled_mixer=use_controlled_mixer,
                                              control_qubit=control_qubit,
                                              use_manual=use_manual)
    
    print(f"\nProceeding with sampling...")
    
    # Sample from optimized state
    print("Sampling from optimized state...")
    counts = sample_final_results(qaoa_ansatz, result.x, shots=1000)
    
    # Analyze top solutions
    print("\nTop 10 most frequent solutions:")
    sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    
    for i, (bitstring, count) in enumerate(sorted_counts[:10]):
        probability = count / 1000
        actual_cost = evaluate_bitstring_cost(bitstring, ising_op)
        print(f"{i+1:2d}. {bitstring}: {count:3d} counts ({probability:.1%}), cost = {actual_cost:.1f}")
    
    # Find the best solution by actual cost
    print("\nBest solutions by actual cost:")
    cost_sorted = [(bs, cnt, evaluate_bitstring_cost(bs, ising_op)) 
                   for bs, cnt in sorted_counts]
    cost_sorted.sort(key=lambda x: x[2])  # Sort by cost
    
    for i, (bitstring, count, cost) in enumerate(cost_sorted[:30]):
        probability = count / 1000
        print(f"{i+1}. {bitstring}: cost = {cost:.1f}, frequency = {count} ({probability:.1%})")
    
    # Analysis of control qubit behavior (if using controlled mixer)
    if use_controlled_mixer:
        print(f"\n{'='*40}")
        print(f"CONTROL QUBIT ANALYSIS (q{control_qubit})")
        print(f"{'='*40}")
        
        control_0_solutions = []
        control_1_solutions = []
        
        for bitstring, count in sorted_counts:
            if bitstring[control_qubit] == '0':
                control_0_solutions.append((bitstring, count))
            else:
                control_1_solutions.append((bitstring, count))
        
        total_control_0 = sum(count for _, count in control_0_solutions)
        total_control_1 = sum(count for _, count in control_1_solutions)
        
        print(f"Solutions with q{control_qubit}=|0⟩ (control active): {total_control_0} ({total_control_0/10:.1f}%)")
        print(f"Solutions with q{control_qubit}=|1⟩ (control inactive): {total_control_1} ({total_control_1/10:.1f}%)")
        
        print(f"\nTop solutions when q{control_qubit}=|0⟩ (control active):")
        for i, (bitstring, count) in enumerate(control_0_solutions[:5]):
            cost = evaluate_bitstring_cost(bitstring, ising_op)
            print(f"  {bitstring}: cost = {cost:.1f}, frequency = {count}")
        
        print(f"\nTop solutions when q{control_qubit}=|1⟩ (control inactive):")  
        for i, (bitstring, count) in enumerate(control_1_solutions[:5]):
            cost = evaluate_bitstring_cost(bitstring, ising_op)
            print(f"  {bitstring}: cost = {cost:.1f}, frequency = {count}")
    
    return result, sorted_counts


# Run the controlled mixer version with your x_control_0[0] variable
print("Running controlled QAOA where x_control_0[0] (q6) controls all other variables...")
optimization_result, top_solutions = complete_qaoa_workflow(
    ising_op, 
    reps=1, #change this to 1 if you want to use only one layer and visualize the circuit
    use_controlled_mixer=True, 
    control_qubit=6,  # x_control_0[0] is qubit 6, you have to change this if you want to use a different control qubit from the SparsePauliOp
    use_manual=True   # Use clean manual implementation
)

Running controlled QAOA where x_control_0[0] (q6) controls all other variables...

QAOA WORKFLOW
Configuration:
  - QAOA layers (reps): 1
  - Use controlled mixer: True
  - Use manual implementation: True
  - Control qubit: q6 (x_control_0[0])
  - Control logic: RX rotations when q6 = |0⟩

Using MANUAL controlled mixer: q6 controls all other qubits (activates when q6=|0⟩)
Creating manual controlled QAOA ansatz:
  - Control qubit: q6 (activates when |0⟩)
  - Target qubits: [0, 1, 2, 3, 4, 5, 7, 8, 9]
  - QAOA layers: 1
Manual ansatz created with 2 parameters
QAOA circuit has 2 parameters
Circuit depth: 37
Number of qubits: 10
Manual controlled circuit created! Drawing...
Text representation:
     ┌───┐ Initial |+⟩ ┌───────────────┐                                   »
q_0: ┤ H ├──────░──────┤ Rz(14.0*γ[0]) ├──■────────────────────────────────»
     ├───┤      ░      ├───────────────┤  │                                »
q_1: ┤ H ├──────░──────┤ Rz(28.0*γ[0]) ├──┼────────────────■─────────